In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from keras_preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array

from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import ResNet50

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
df['labels'] = df['labels'].apply(lambda s: s.split(' '))
df[:10]

In [ ]:
HEIGHT = 224
WIDTH=224
SEED = 40
BATCH_SIZE=32

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1/255.,
    validation_split= 0.2
    )

In [ ]:
train_ds = datagen.flow_from_dataframe(
    df,
    directory = '../input/resized-plant2021/img_sz_512',
    x_col = 'image',
    y_col = 'labels',
    subset="training",
    color_mode="rgb",
    target_size = (HEIGHT,WIDTH),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=SEED,
)

In [ ]:
val_ds = datagen.flow_from_dataframe(
    df,
    directory = '../input/resized-plant2021/img_sz_512',
    x_col = 'image',
    y_col = 'labels',
    subset="validation",
    color_mode="rgb",
    target_size = (HEIGHT,WIDTH),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=SEED,
)

In [ ]:
pretrained_model = ResNet50(input_shape=(HEIGHT,WIDTH,3), include_top=False, weights='../input/resnet50-weight/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
#fully connected layer
x = Dense(64, activation='relu')(x)
x = Dense(16, activation='relu')(x)
# finally, the softmax for the classifier 
x = Dense(6, activation='softmax')(x)
model = Model(pretrained_model.input, x)

# Compile the Model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, decay=0.01/10, momentum=0.8, nesterov=True),
    loss='binary_crossentropy',
    metrics=['accuracy'])

model.summary()

In [ ]:
checkpoint=ModelCheckpoint(r'model\model_ResNet50.h5',
                          monitor='val_loss',
                          mode='min',
                          save_best_only=True,
                          verbose=1)
earlystop=EarlyStopping(monitor='val_loss',
                       min_delta=0,
                       patience=5,
                       verbose=1,
                       restore_best_weights=True)

callbacks=[checkpoint,earlystop]

In [ ]:
resNet_model=model.fit_generator(train_ds,
                        validation_data=val_ds,
                        epochs=30,
                        shuffle=True,
                        callbacks=callbacks)


In [ ]:
model.save('resnet50_model.h5')